# TikTok Virality Prediction Project

- **Juan López Segura**
- **Ignacio Urretavizcaya Tato**

This Jupyter Notebook details a project focused on separately analyzing audio, text, and video data extracted from 1,000 trending TikTok videos of 2021. The aim is to utilize these multimodal data inputs either by integrating them into multimodal models or by stacking models to predict the virality of TikTok videos.

## The Work

### Data Collection

The data for this project is organized into separate folders based on the type of data:

- Audio data: `data/inputs/audio/`
- Text data: `data/inputs/text/`
- Video data: `data/inputs/videos/`

### Data Description

Each type of data includes elements specific to the medium:
- **Audio:** Includes the audio tracks from the videos.
- **Text:** Consists of hashtags, video text descriptions, and transcriptions of the audio.
- **Video:** Contains the video files themselves. For the analysis we will take 10 frames from each video.

## Purpose

The purpose of this analysis is to predict the virality of TikTok videos based on a defined metric of virality, which includes views, comments, likes, etc. By analyzing these metrics and their correlations, we can develop a predictive model.

### Virality Metric Definition

#### Data Loading and Initial Processing
